<a href="https://colab.research.google.com/github/Saisathvik-Achanta/Natural-Language-Processing/blob/main/NLP_ASSIGNMENT_07_2203A51330.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install tensorflow numpy

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [13]:
data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]
english_texts, french_texts = zip(*data)
french_texts = ["start " + text + " end" for text in french_texts]
def tokenize(sentences):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    return tokenizer, tokenizer.texts_to_sequences(sentences)

eng_tokenizer, eng_sequences = tokenize(english_texts)
fr_tokenizer, fr_sequences = tokenize(french_texts)
max_eng_len = max([len(seq) for seq in eng_sequences])
max_fr_len = max([len(seq) for seq in fr_sequences])

eng_sequences = pad_sequences(eng_sequences, maxlen=max_eng_len, padding="post")
fr_sequences = pad_sequences(fr_sequences, maxlen=max_fr_len, padding="post")
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fr_vocab_size = len(fr_tokenizer.word_index) + 1

eng_train, eng_val, fr_train, fr_val = train_test_split(eng_sequences, fr_sequences, test_size=0.2)

In [14]:
embedding_dim = 256
latent_dim = 512
encoder_inputs = Input(shape=(max_eng_len,))
encoder_embedding = Embedding(eng_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(max_fr_len,))
decoder_embedding = Embedding(fr_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(fr_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [15]:
fr_train_target = np.delete(fr_train, 0, axis=1)
fr_train_target = np.insert(fr_train_target, max_fr_len - 1, 0, axis=1)
fr_val_target = np.delete(fr_val, 0, axis=1)
fr_val_target = np.insert(fr_val_target, max_fr_len - 1, 0, axis=1)

In [16]:
fr_train_target = np.delete(fr_train, 0, axis=1)
fr_train_target = np.insert(fr_train_target, max_fr_len - 1, 0, axis=1)
fr_val_target = np.delete(fr_val, 0, axis=1)
fr_val_target = np.insert(fr_val_target, max_fr_len - 1, 0, axis=1)


In [17]:
batch_size = 64
epochs = 100

history = model.fit(
    [eng_train, fr_train],
    fr_train_target[..., np.newaxis],
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([eng_val, fr_val], fr_val_target[..., np.newaxis]),
)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.0000e+00 - loss: 2.7740 - val_accuracy: 0.3000 - val_loss: 2.7384
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step - accuracy: 0.4000 - loss: 2.7134 - val_accuracy: 0.3000 - val_loss: 2.6958
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.4000 - loss: 2.6402 - val_accuracy: 0.3000 - val_loss: 2.6269
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.4000 - loss: 2.5324 - val_accuracy: 0.3000 - val_loss: 2.5091
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.4000 - loss: 2.3617 - val_accuracy: 0.3000 - val_loss: 2.3221
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - accuracy: 0.4000 - loss: 2.1003 - val_accuracy: 0.3000 - val_loss: 2.1338
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step - accuracy: 0.4000 - loss: 1.7999 - val_accuracy: 0.3000 - val_loss: 2.2223
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.4000 - loss: 1.6999 - val_accuracy: 0.3000 

In [18]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [19]:
def translate_sentence(sentence):
    sequence = eng_tokenizer.texts_to_sequences([sentence])
    sequence = pad_sequences(sequence, maxlen=max_eng_len, padding="post")

    states = encoder_model.predict(sequence)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = fr_tokenizer.word_index["start"]

    translated_sentence = ""
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = fr_tokenizer.index_word.get(sampled_token_index, "")

        if sampled_word == "end":
            stop_condition = True
        else:
            translated_sentence += " " + sampled_word
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states = [h, c]

    return translated_sentence.strip()
print(translate_sentence("hello"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
bonjour
